In [ ]:
import cv2
import numpy as np
from flask import Flask, Response

app = Flask(__name__)
print("GO FIRE WALL IS OFF")
# Set up OpenCV face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    return frame

# Set up video stream from ESP32
def get_video_stream():
    import socket
    import io
    import struct

    server_socket = socket.socket()
    server_socket.bind(('localhost', 8000))
    server_socket.listen(0)
    connection = server_socket.accept()[0].makefile('rb')
    try:
        while True:
            # Read the length of the image as a 32-bit unsigned int. If the
            # length is zero, quit the loop
            image_len = struct.unpack('<L', connection.read(struct.calcsize('<L')))[0]
            if not image_len:
                break
            # Construct a stream to hold the image data and read the image
            # data from the connection
            image_stream = io.BytesIO()
            image_stream.write(connection.read(image_len))
            # Rewind the stream, open it as an image with PIL and do some
            # processing on it
            image_stream.seek(0)
            frame = np.frombuffer(image_stream.getvalue(), dtype=np.uint8)
            frame = cv2.imdecode(frame, cv2.IMREAD_COLOR)
            frame = detect_faces(frame)
    finally:
        connection.close()
        server_socket.close()

@app.route('/')
def index():
    return Response(get_video_stream(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='192.168.0.102', port=8000, debug=False) #dit was via 0.0.0.0 maar die pakt alle interfaces
